In [1]:
import os
import sys
import pprint
import glob
import imp
import subprocess

from IPython import display
import chardet

import climatools.misc as misc

imp.reload(misc)


<module 'climatools.misc' from '/nuwa_cluster/home/jackyu/climatools/climatools/misc.py'>

## Lowercase whole source code

The original LBLRTM code is written in Fortran 77, and all in block capitals!  Here, it is converted to lowercase.  Maybe this will make it more legible.

In [2]:
def dir_src_lblrtm():
    dir_lblrtm = os.path.join('/nuwa_cluster/home/jackyu', 
                              'line_by_line/aerlbl_v12.2_package',
                              'lblrtm/src')
    return dir_lblrtm


def fortran_files():
    dir_lblrtm = dir_src_lblrtm()
    
#    files = [name for name in os.listdir(dir_lblrtm) 
#             if name.endswith('.f90')]
    files = ['lblrtm.f90',]
    paths = [os.path.join(dir_lblrtm, file) for file in files]
    return paths

In [3]:
def detect_encoding(readfrom=None):
    '''
    Use the *chardet* module to determine the encoding
    of a file.
    
    Parameters
    ----------
    readfrom: path to file whose encoding is to be determined
    '''
    proc = subprocess.Popen(['chardetect', readfrom], 
                            stdout=subprocess.PIPE)
    out = proc.communicate()[0]
    encoding = out.decode('utf-8').split()[1]
    return encoding



def get_filecontent(readfrom=None):
    '''
    Determine encoding of file first, then open it, read it
    and return its content
    
    Parameters
    ----------
    readfrom: path to file whose content is to be returned
    '''
    encoding = detect_encoding(readfrom=readfrom)
    with open(readfrom, mode='r', encoding=encoding) as f:
        c = f.read()
    return c



def lowercase_fortran(readfrom=None, chname='lowercase'):
    '''
    Turn all content of file into lowercase
    
    Parameters
    ----------
    readfrom: path to file whose content is to be turned into lowercase
    '''
    basename = os.path.basename(readfrom)
    filename, extension = os.path.splitext(basename)
    c = get_filecontent(readfrom=readfrom)
    
    if chname:
        name_insert = '_{}'.format(chname)
    else:
        name_insert = ''
        
    with open(filename + name_insert + extension, 
              mode='w', encoding='utf-8') as f:
        f.write(c.lower())


In [4]:
os.chdir(dir_src_lblrtm())

In [5]:
for name in os.listdir():
    if name.endswith('.f90'):
        print(name)
        lowercase_fortran(readfrom=name, chname='')

fftscn.f90
util_linux_pgi.f90
ncargks.f90
util_gfortran.f90
lbllow.f90
contnm.f90
util_aix.f90
util_OS_X_absoft.f90
planet_mars.f90
planet_earth.f90
lblatm.f90
phys_consts.f90
util_OS_X_ifort.f90
util_sgi.f90
xmerge.f90
pltlbl.f90
util_OS_X_ibm_xl.f90
util_linux_intel.f90
lblrtm.f90
util_cray.f90
util_dos.f90
postsub.f90
nonlte.f90
oprop.f90
util_hp.f90
fftscn_dbl.f90
util_alpha.f90
struct_types.f90
testmm.f90
lblparams.f90
util_sun.f90
util_OS_X_g95.f90
lbldum.f90
solar.f90


## LBLRTM main program

In [18]:
def dir_example():
    directory = os.path.join('/nuwa_cluster/home/jackyu',
                             'line_by_line/aerlbl_v12.2_package', 
                             'runs/LW',
                             'run_mls70pro_H2O_cont')
    return directory



def get_signposts(readfrom='out.log'):
    '''
    Return the set of unique tuples of (signpost number, statement)
    that are evaluated in the lblrtm run.  From this set, it can be
    seen if certain signpost numbers or statements are reused;
    each signpost number should have a unique statement.
    
    Parameters
    ----------
    readfrom: path to file which contains the output of `write`
              statements which have been placed in the source code
              as markers
    '''
    with open(readfrom, mode='r', encoding='utf-8') as f:
        lines = f.readlines()
        
    lines = [l.strip() for l in lines 
             if l.strip().startswith('signpost')]
    
    lst_sgnpost_sgn = [l.split(' ', 1) for l in lines]
    
    lst_sgnpost_sgn = [(int(sgnpost
                            .split('signpost')[1]
                            .split('.')[0]),
                        sgn)
                       for sgnpost, sgn in lst_sgnpost_sgn]
    
    return set(lst_sgnpost_sgn)

The LBLRTM main program contains numerous `if` statements and statement labels (e.g. `10 continue`).  Depending on the type of calculation specified by the input parametres in TAPE5, not all of these are executed in each LBLRTM run.  By placing `write` statements that write to the standard output just *inside* all top-level `if` statements, or immediately after *continue* statements, it is possible to determine the ones that a particular LBLRTM run goes through (or evaluates).

For a typical longwave flux calculation run that I have been using, these are all the unique `if` and `continue` statements that are evaluated.  Some of them are evaluated more than once.  To see the exact sequence in which they are evaluated, see the 'out.log' file that is created when the `write` statements are placed in the source code.  

In [16]:
get_signposts(readfrom=os.path.join(dir_example(), 'out.log'))

{(1, 'elseif (icntnm.eq.1) then'),
 (2, 'if (ihirac.gt.0) then'),
 (3, 'else read (cmrg(2),930) imrg'),
 (4, 'if (ihirac.eq.1.or.ihirac.eq.4.or.ilblf4.ge.1) then'),
 (5, 'if (icntnm.ne.0) then'),
 (6, 'if ((ihirac+iaersl+iemit+iatm+ilas).gt.0) then'),
 (7, '40 continue'),
 (8, 'if (iod.eq.0) then'),
 (9, 'if (ihirac+iatm+imrg.gt.0) then'),
 (10, 'if (imrg.eq.1) then'),
 (11, '60 continue'),
 (12, '70 continue'),
 (13, 'if (icntnm.eq.0) then'),
 (14, 'if (mmrg.ge.3) then'),
 (15, 'if (iemit.gt.0) then'),
 (16, 'if ((mmrg.ge.3).and.(mmrg.ne.9)) then'),
 (17, 'if (cxida(1).eq.cprcnt) then'),
 (18, '90 conitnue')}

In [17]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')